# Testing Environment for Eurus
Shaun Hadden & Brendan Smithyman | October, 2015

## Imports

In [ ]:
import numpy as np
from extend import Eurus

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

%matplotlib inline

## Helper functions

In [ ]:
class Source(object):
    
    def __init__(self, sc):
                
        self._x, self._z = np.mgrid[
            0:sc['dx']*sc['nx']:sc['dx'],
            0:sc['dz']*sc['nz']:sc['dz']
        ]
    
    def __call__(self, x, z):
        
        dist = np.sqrt((self._x - x)**2 + (self._z - z)**2)
        srcterm = 1.*(dist == dist.min())
        nz, nx = self._x.shape
        
        return np.hstack([srcterm.ravel() / srcterm.sum(), np.zeros(nx*nz, dtype=np.complex128)])

## Modelling setup

In [ ]:
# Geometry parameters
dx          = 1.
dz          = 1.
nx          = 100
nz          = 200

# Bulk parameters
velocity    = 2000.     * np.ones((nz,nx))
density     = 2700.     * np.ones((nz,nx))

# Anisotropy parameters
theta       = 0.5*np.pi * np.ones((nz,nx))
epsilon     = 0.        * np.ones((nz,nx))
delta       = 0.        * np.ones((nz,nx))

# Other parameters
freq        = 100.
freeSurf    = [False, False, False, False]
nPML        = 10


# Pack values into systemConfig dictionary
systemConfig = {
    'nx':       nx,
    'nz':       nz,
    'dx':       dx,
    'dz':       dz,

    'c':        velocity,
    'rho':      density,
    
    'theta':    theta,
    'eps':      epsilon,
    'delta':    delta,

    'freq':     freq,
    'freeSurf': freeSurf,
    'nPML':     nPML,
}

## Testing

In [ ]:
Ainv = Eurus(systemConfig)
src = Source(systemConfig)

In [ ]:
plt.spy(Ainv.A, markersize=0.1, marker=',')

In [ ]:
q = src(15,15)
u = Ainv*q

In [ ]:
plotopts = {
    'extent':   [0, nx*dx, nz*dz, 0],
    'cmap':     cm.bwr,
#     'vmin':     -1.,
#     'vmax':     1.,
}

fig = plt.figure()

ax = fig.add_subplot(1,1,1, aspect=1)
plt.imshow(u[:nx*nz].reshape((nz,nx)).real, **plotopts)
plt.title('Real Wavefield')
plt.xlabel('X')
plt.ylabel('Z')